In [ ]:
!jt -t grade3 -fs 13 -nf opensans -nfs 13 -N -kl -cursw 5 -cursc r -cellw 1800 -T -ofs 13 


In [1]:
import pandas as pd
import numpy as np

In [4]:
 df = pd.read_csv("a_file")

In [5]:
df

,Train/Test,Author,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,Train,bailey-s,0.159715,0.093428,-0.044837,0.012579,0.028296,0.037956,0.029422,0.023161,...,0.015426,0.003035,0.018192,-0.007347,-0.034160,0.007488,0.017504,0.040404,-0.029948,-0.001714
1,Train,bailey-s,0.124973,0.076745,-0.033624,0.013994,0.013203,0.027903,0.016636,0.015735,...,0.021322,-0.012360,0.041797,0.025950,-0.004951,0.011476,0.026273,0.045907,0.019079,0.008144
2,Train,bailey-s,0.155616,0.041525,-0.020922,0.003896,0.068121,0.036189,0.008566,0.022201,...,-0.006359,0.004180,-0.009600,-0.025567,-0.014735,0.009266,0.000668,0.013842,-0.022312,-0.024612
3,Train,bailey-s,0.118614,0.077235,-0.035256,0.007047,0.014933,0.036521,0.012629,0.026490,...,-0.025813,0.005341,-0.035938,0.057491,0.037418,-0.098250,0.001497,-0.095737,0.031895,0.051032
4,Train,bailey-s,0.158510,0.117368,-0.054584,0.009151,-0.002736,0.036624,0.028879,0.035479,...,-0.015448,-0.002792,-0.044809,0.012338,-0.007852,-0.050123,0.012428,-0.046734,0.002005,0.022012
5,Train,bailey-s,0.124079,0.089007,-0.036675,0.015402,-0.005746,0.034424,0.022953,0.014319,...,0.025518,-0.032789,-0.018014,-0.023673,-0.001165,0.029052,-0.015556,0.010143,0.003236,-0.027732
6,Train,bailey-s,0.381671,0.294416,-0.143992,0.025329,0.018792,0.070683,0.080022,0.074747,...,0.007333,-0.013969,-0.006698,-0.004002,0.015823,-0.002340,0.013815,0.010796,-0.010845,-0.012204
7,Train,bailey-s,0.202022,0.144231,-0.066499,0.015915,-0.008843,0.043012,0.037566,0.035295,...,0.001495,-0.006983,-0.003806,-0.028229,0.009211,0.001213,0.049178,-0.000613,-0.016520,0.006114
8,Train,bailey-s,0.151689,0.089285,-0.043188,0.011982,0.027413,0.036066,0.027918,0.022615,...,0.015929,0.003780,0.030737,-0.001440,-0.044494,0.018866,0.020722,0.041005,-0.029393,-0.002381
9,Train,bailey-s,0.130353,0.087935,-0.041555,0.006895,0.004911,0.032143,0.026682,0.021309,...,0.029381,-0.010014,0.000006,-0.006701,-0.012775,0.020683,0.000642,0.021506,-0.016610,0.002472


In [6]:
train_df = df[df["Train/Test"] == "Train"]

In [ ]:
#train=train_df.iloc[:,1:] #removes "Test/Train column"

In [7]:
train_df

,Train/Test,Author,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,Train,bailey-s,0.159715,0.093428,-0.044837,0.012579,0.028296,0.037956,0.029422,0.023161,...,0.015426,0.003035,0.018192,-0.007347,-0.034160,0.007488,0.017504,0.040404,-0.029948,-0.001714
1,Train,bailey-s,0.124973,0.076745,-0.033624,0.013994,0.013203,0.027903,0.016636,0.015735,...,0.021322,-0.012360,0.041797,0.025950,-0.004951,0.011476,0.026273,0.045907,0.019079,0.008144
2,Train,bailey-s,0.155616,0.041525,-0.020922,0.003896,0.068121,0.036189,0.008566,0.022201,...,-0.006359,0.004180,-0.009600,-0.025567,-0.014735,0.009266,0.000668,0.013842,-0.022312,-0.024612
3,Train,bailey-s,0.118614,0.077235,-0.035256,0.007047,0.014933,0.036521,0.012629,0.026490,...,-0.025813,0.005341,-0.035938,0.057491,0.037418,-0.098250,0.001497,-0.095737,0.031895,0.051032
4,Train,bailey-s,0.158510,0.117368,-0.054584,0.009151,-0.002736,0.036624,0.028879,0.035479,...,-0.015448,-0.002792,-0.044809,0.012338,-0.007852,-0.050123,0.012428,-0.046734,0.002005,0.022012
5,Train,bailey-s,0.124079,0.089007,-0.036675,0.015402,-0.005746,0.034424,0.022953,0.014319,...,0.025518,-0.032789,-0.018014,-0.023673,-0.001165,0.029052,-0.015556,0.010143,0.003236,-0.027732
6,Train,bailey-s,0.381671,0.294416,-0.143992,0.025329,0.018792,0.070683,0.080022,0.074747,...,0.007333,-0.013969,-0.006698,-0.004002,0.015823,-0.002340,0.013815,0.010796,-0.010845,-0.012204
7,Train,bailey-s,0.202022,0.144231,-0.066499,0.015915,-0.008843,0.043012,0.037566,0.035295,...,0.001495,-0.006983,-0.003806,-0.028229,0.009211,0.001213,0.049178,-0.000613,-0.016520,0.006114
8,Train,bailey-s,0.151689,0.089285,-0.043188,0.011982,0.027413,0.036066,0.027918,0.022615,...,0.015929,0.003780,0.030737,-0.001440,-0.044494,0.018866,0.020722,0.041005,-0.029393,-0.002381
9,Train,bailey-s,0.130353,0.087935,-0.041555,0.006895,0.004911,0.032143,0.026682,0.021309,...,0.029381,-0.010014,0.000006,-0.006701,-0.012775,0.020683,0.000642,0.021506,-0.016610,0.002472


In [8]:
test_df = df[df["Train/Test"] == "Test"]

In [10]:
test_df.reset_index(inplace=True, drop=True)
test_df

,Train/Test,Author,0,1,2,3,4,5,6,7,...,290,291,292,293,294,295,296,297,298,299
0,Test,salisbury-h,0.151015,0.047998,-0.035866,0.008835,0.043910,0.105798,0.008606,0.449704,...,-0.015284,0.002797,-0.001918,-0.025861,0.014545,0.024086,-0.051347,0.023368,-0.041234,-0.039404
1,Test,corman-s,0.150082,0.129109,-0.100882,0.178743,0.031859,-0.043244,-0.046689,-0.002104,...,-0.033239,0.009078,0.010465,0.013434,0.092433,-0.015556,0.009644,0.010664,0.004541,-0.064871
2,Test,mccarty-d,0.212019,0.125014,-0.054901,0.070007,0.025352,0.172979,0.313801,-0.031651,...,0.003238,0.009593,-0.003026,0.007825,0.029598,0.052581,-0.009374,0.013630,0.014424,-0.004961
3,Test,lenhart-m,0.712027,-0.398689,0.055934,-0.036145,0.420948,0.021989,-0.008748,0.003664,...,-0.003296,-0.005924,-0.002115,0.010346,0.003768,-0.003408,-0.002748,-0.000301,0.002764,-0.006299
4,Test,corman-s,0.112653,0.065292,-0.050617,0.086484,0.022388,-0.011457,-0.028439,0.003441,...,-0.013408,-0.012568,-0.004003,-0.024968,0.008206,-0.054193,0.013948,0.026232,0.013878,-0.002879
5,Test,quigley-d,0.138127,0.147356,0.124644,-0.015322,0.002239,-0.007432,-0.008799,0.000960,...,-0.016350,0.012865,0.113344,-0.006443,0.042745,-0.021399,-0.031395,0.007243,0.007899,0.061833
6,Test,lenhart-m,0.749476,-0.410794,0.058462,-0.033540,0.397071,0.020794,-0.011053,0.002797,...,-0.003590,0.008207,-0.001534,0.000418,0.003882,-0.003849,-0.007921,0.002001,-0.003250,-0.004300
7,Test,keiser-k,0.135107,0.142541,-0.218422,-0.230862,0.036943,-0.021792,-0.013322,-0.021085,...,0.009430,0.029664,-0.004780,-0.001559,-0.001613,-0.004846,0.013167,0.003656,0.005157,0.013431
8,Test,lenhart-m,0.437565,-0.164526,-0.015000,-0.003426,-0.244551,-0.035030,0.014630,-0.008977,...,0.000655,0.004315,0.010042,-0.037635,-0.000961,0.008371,0.025952,0.019075,-0.009919,-0.027738
9,Test,corman-s,0.159311,0.149754,-0.121362,0.215244,0.027661,-0.048672,-0.035736,-0.012181,...,0.005165,0.014274,0.019952,0.001702,0.022219,0.001183,-0.003521,0.051686,-0.023197,-0.012921


In [11]:
#get author name
d1_df = df.iloc[2928]
d1_df[1] 

'quigley-d'

In [12]:
#also get author name
d1_df = df[df.index == 2928]
author1 = d1_df['Author']
author1 = author1.values[0] 

allt ovan är bara läsa in df i två olika df's => test och train

In [13]:
import torch
import random
from torch.autograd import Variable

In [14]:
batch_size = 1
for i in range(batch_size):
    print(i)

0


In [ ]:
#df_filename = df.drop("Filename", axis=1) 

In [17]:
samples = []
tensor_label_pairs = []
for i in range(batch_size):
    authors = list(set(df["Author"]))
    seed_authors = random.choices(authors, k=2)
    data_train = df[df["Train/Test"] == "Train"]
    seed_entries_1 = (data_train[data_train["Author"] == seed_authors[0]]).sample(n=2, replace=True)
    #print(seed_entries_1.iloc[0])
    seed_entries_2 = (data_train[data_train["Author"] == seed_authors[1]]).sample(n=2, replace=True)
    if random.random() < 0.5:
        samples.append((seed_entries_1.iloc[0], seed_entries_1.iloc[1],0))
    else:
        samples.append((seed_entries_1.iloc[0], seed_entries_2.iloc[0],1))
for sample in samples:
    a = list(sample[0])[2:-2]
    #print(sample[0][1:-2])
   #print(sample[0].values[1:-1])
    b = list(sample[1])[2:-2]
    c = Variable(torch.Tensor((a+b)))
    label = torch.Tensor([sample[2]])
    print(sample[2])
    tensor_label_pairs.append((c,label))
print(tensor_label_pairs)


0
[(tensor([ 7.4151e-02,  5.2922e-02, -1.5066e-02,  1.9609e-02, -9.9758e-03,
         2.0321e-02,  4.8071e-03,  1.2908e-02,  1.9406e-02, -3.4862e-03,
         6.0154e-03,  9.9475e-03,  3.3659e-02,  6.0982e-02, -6.4204e-02,
        -6.5402e-02,  2.3953e-02, -7.3181e-02,  2.3621e-01, -2.7650e-02,
         1.6248e-01,  8.8251e-02, -5.1772e-02,  2.7656e-02, -6.6478e-02,
        -5.2861e-02,  8.2600e-03,  1.0551e-02, -2.8937e-02,  2.1122e-03,
        -1.0844e-02, -6.1440e-03, -1.2141e-02, -2.6672e-02,  8.5337e-03,
         2.6778e-02,  8.0588e-04, -3.7348e-03, -1.4101e-02,  1.4560e-02,
         1.4711e-03, -1.3366e-02, -2.0860e-02, -3.5718e-02, -3.0811e-02,
        -4.6910e-03, -2.0875e-02,  1.1819e-02,  7.7623e-03,  3.8118e-02,
         7.8045e-03, -7.3526e-03, -2.8010e-02,  2.2320e-04,  4.9129e-02,
         9.3625e-03, -1.6347e-02,  1.8675e-03,  5.0207e-03, -2.7326e-02,
        -1.5302e-02, -8.3597e-03,  9.5197e-04,  3.1636e-03,  9.0123e-04,
         9.0567e-03, -1.9703e-02,  5.6609e-03, 

In [18]:
authors = df.Author.unique().tolist()
print(len(authors))
first_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(first_author)
second_author = authors.pop(random.randrange(0,(len(authors))))
print(len(authors))
print(second_author)

14
13
saibi-e
12
corman-s


In [43]:
df1 = df[df["Author"].values == "bailey-s"]

row = np.random.choice(df.index.values) # random int picking the row out of all available rows
sampled_row = df.iloc[row] # the random row picked incl "Train/Test" and "Author", type 'pandas.core.series.Series' 
sampled = sampled_row[2:] # random row picked without "Train/Test" and "Author", type 'pandas.core.series.Series'
sampled_list = sampled.tolist() # random row as list
sampled_tensor = torch.Tensor(sampled_list) # random row as torch.Tensor

sampled_tensor_oneliner = torch.Tensor((sampled_df[2:]).tolist()) # lines 5,6,7 as one



In [69]:
one_row = df[df["Author"].values == first_author].sample(n=1) # pandas df w one randomly picked row, incl "Train/Test" and "Author"
one_row_minus = one_row.drop(["Train/Test", "Author"], axis=1) # pandas df w one randomly picked row, EXCL "Train/Test" and "Author"
one_values = one_row_minus.values[0] # numpy.ndarray w one randomly picked row, incl "Train/Test" and "Author"
one_torch = torch.from_numpy(one_values) # torch.Tensor w one randomly picked row, incl "Train/Test" and "Author"


one_oneliner = torch.from_numpy(df[df["Author"].values == first_author].sample(n=1).drop(["Train/Test", "Author"], axis=1).values) # lines 1, 2, 3, 4 as one

In [ ]:
x = torch.randn(2, 3)
print(x)
y = torch.randn(2, 3)
print(y)
z = torch.randn(2, 3)
print(z)
torch.cat((x, y, z), 0)

In [ ]:

def sample_data(size, df):
    def compare_author(i1, i2):
        d1_df = df[df.index == i1]
        d2_df = df[df.index == i2]
        author1 = d1_df['0']
        a1 = author1.values.tolist()
        author2 = d2_df['0']
        a2 = author2.values.tolist()
        d1 = d1_df.values.tolist()
        d1 = d1[0][3:]
        d2 = d2_df.values.tolist()
        d2 = d2[0][3:]
        if a1 == a2:
            c = 1
        else:
            c = 0
        d = d1 + d2
        sample = (d, c)
        return sample

    samples = []
    counter1 = 0
    counter0 = 0
    while len(samples) <= size:
        index1 = random.choice(df.index)
        index2 = random.choice(df.index)
        while index1 == index2:
            index2 = random.choice(df.index)
        sample = compare_author(index1, index2)
        if sample[1] == 1:
            if counter1 <= size/2 + 1:
                samples.append(sample)
                counter1 += 1
        else:
            if counter0 <= size/2 + 1:
                samples.append(sample)
                counter0 += 1
    random.shuffle(samples)
    print(samples)

In [ ]:
sample_data(5, train_df)